In [1]:
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from time import time

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

INFO (theano.gof.compilelock): Waiting for existing lock by process '18252' (I am process '23734')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ec2-user/.theano/compiledir_Linux-4.4-amzn1.x86_64-x86_64-with-glibc2.2.5-x86_64-3.6.1-64/lock_dir
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


In [2]:
from keras.optimizers import Adam

In [3]:
start_time_0 = time()

### Load file

In [4]:
lyrics = open('BSB.txt').read()
lyrics = lyrics.lower()

### Convert char to num

In [5]:
chars = [i for i in sorted(set(lyrics))]

In [6]:
char_num = dict((char, num) for num, char in enumerate(chars))
num_char = dict((num, char) for num, char in enumerate(chars))

In [7]:
lyrics_num = [char_num[i] for i in lyrics]


In [8]:
alphabet = len(char_num)

### create sequence

In [9]:
def chop_to_sequence(seq, lyrics_num):
    """
    chop lyrics_num into segments with length seq
    return list of segments
    """
    lyrics_arr = np.asarray(lyrics_num)
    lyrics_arr = lyrics_arr / float(alphabet)
    segments = []
    next_char = []
    for i in range(0, len(lyrics_num)-seq):
        segment = lyrics_arr[i:i+seq]
        segments.append(segment)
        next_char.append(lyrics_num[i+seq])

    print("segment length:", seq)
    print('number of segments:', len(segments))
    print("chars in lyrics:", len(lyrics))
    print("")
    
    segments = np.reshape(segments, (len(segments),seq,1))
    next_char = np_utils.to_categorical(next_char)
    
    return segments, next_char
    

In [10]:
def print_time(start_time):
    print((time()-start_time)/60)

In [11]:
start_time = time()
X_all, y_all = chop_to_sequence(30, lyrics_num)
print_time(start_time)


segment length: 30
number of segments: 36488
chars in lyrics: 36518

0.0007930159568786621


### LSTM model

In [12]:
def predict_next_n(n):
    """
    predict next n char from random seed
    """
    start = np.random.randint(0, len(X))
    seed = lyrics[start:start+X.shape[1]]
    pattern = X[start]
    
    chars=[]
    for i in range(n):
        pred_num = model.predict_classes(np.reshape(pattern,(1,pattern.shape[0],1)), verbose=False)
        #pred_num = np.argmax(pred_arr)
        pred_char = num_char[pred_num[0]]
        
        chars.append(pred_char)
        pattern = np.append(pattern, pred_num/float(alphabet))
        pattern = pattern[1:]
        #print(pred_arr, pred_num, pred_char, pattern)
    print("Seed:", seed)
    print("Generated:", "".join(chars))
    

### All data

In [13]:
X=X_all
y=y_all

In [14]:
X[0], y[0]

(array([[ 0.61538462],
        [ 0.94871795],
        [ 0.76923077],
        [ 0.53846154],
        [ 0.38461538],
        [ 0.79487179],
        [ 0.        ],
        [ 0.30769231],
        [ 0.30769231],
        [ 0.30769231],
        [ 0.30769231],
        [ 0.        ],
        [ 0.07692308],
        [ 0.53846154],
        [ 0.02564103],
        [ 0.8974359 ],
        [ 0.33333333],
        [ 0.66666667],
        [ 0.66666667],
        [ 0.33333333],
        [ 0.02564103],
        [ 0.35897436],
        [ 0.43589744],
        [ 0.02564103],
        [ 0.8974359 ],
        [ 0.53846154],
        [ 0.82051282],
        [ 0.51282051],
        [ 0.02564103],
        [ 0.94871795]]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]))

In [27]:
y[1]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.])

### Create Model

In [20]:
model = Sequential()
model.add(LSTM(alphabet, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(alphabet))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30, 39)            6396      
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 39)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 39)                12324     
_________________________________________________________________
dropout_3 (Dropout)          (None, 39)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 39)                1560      
Total params: 20,280.0
Trainable params: 20,280.0
Non-trainable params: 0.0
_________________________________________________________________


In [23]:
adam_lr = Adam(lr=0.001)

model.compile(loss='categorical_crossentropy', optimizer=adam_lr, metrics=['accuracy'])
start_time = time()
model.fit(X, y, epochs=2, batch_size=512, verbose= True, validation_split=0.2)

Train on 29190 samples, validate on 7298 samples
Epoch 1/2
29190/29190 [==============================] - 20s - loss: 3.3526 - acc: 0.1295 - val_loss: 3.0715 - val_acc: 0.1750
Epoch 2/2
29190/29190 [==============================] - 20s - loss: 3.1460 - acc: 0.1549 - val_loss: 3.0450 - val_acc: 0.1750


### Takes too long! reduce data to BSB lyrics

In [ ]:
#time to finish 2000 epochs (in hours)
(20*2000)/(60*60)

In [ ]:
model.fit(X, y, epochs=500, batch_size=512, verbose= True, validation_split=0.2)

Train on 29190 samples, validate on 7298 samples
Epoch 1/500
29190/29190 [==============================] - 20s - loss: 3.1117 - acc: 0.1571 - val_loss: 3.0316 - val_acc: 0.1750
Epoch 2/500
29190/29190 [==============================] - 20s - loss: 3.0992 - acc: 0.1586 - val_loss: 3.0306 - val_acc: 0.1750
Epoch 3/500
29190/29190 [==============================] - 20s - loss: 3.0850 - acc: 0.1626 - val_loss: 3.0237 - val_acc: 0.1750
Epoch 4/500
29190/29190 [==============================] - 20s - loss: 3.0794 - acc: 0.1613 - val_loss: 3.0200 - val_acc: 0.1750
Epoch 5/500
29190/29190 [==============================] - 20s - loss: 3.0764 - acc: 0.1640 - val_loss: 3.0205 - val_acc: 0.1750
Epoch 6/500
29190/29190 [==============================] - 20s - loss: 3.0697 - acc: 0.1650 - val_loss: 3.0161 - val_acc: 0.1750
Epoch 7/500
29190/29190 [==============================] - 20s - loss: 3.0623 - acc: 0.1661 - val_loss: 3.0078 - val_acc: 0.1750
Epoch 8/500
29190/29190 [=======================

In [29]:
### save model in between
filepath="P2_weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=1, batch_size=512, callbacks = callbacks_list, verbose= True, validation_split=0.2)

Train on 29190 samples, validate on 7298 samples
Epoch 1/1
29190/29190 [==============================] - 14s - loss: 2.3179 - acc: 0.3215 - val_loss: 2.3047 - val_acc: 0.3317


In [ ]:
model.fit(X, y, epochs=300, batch_size=512, verbose= True, validation_split=0.2)

Train on 29190 samples, validate on 7298 samples
Epoch 1/500
29190/29190 [==============================] - 20s - loss: 3.1117 - acc: 0.1571 - val_loss: 3.0316 - val_acc: 0.1750
Epoch 2/500
29190/29190 [==============================] - 20s - loss: 3.0992 - acc: 0.1586 - val_loss: 3.0306 - val_acc: 0.1750
Epoch 3/500
29190/29190 [==============================] - 20s - loss: 3.0850 - acc: 0.1626 - val_loss: 3.0237 - val_acc: 0.1750
Epoch 4/500
29190/29190 [==============================] - 20s - loss: 3.0794 - acc: 0.1613 - val_loss: 3.0200 - val_acc: 0.1750
Epoch 5/500
29190/29190 [==============================] - 20s - loss: 3.0764 - acc: 0.1640 - val_loss: 3.0205 - val_acc: 0.1750
Epoch 6/500
29190/29190 [==============================] - 20s - loss: 3.0697 - acc: 0.1650 - val_loss: 3.0161 - val_acc: 0.1750
Epoch 7/500
29190/29190 [==============================] - 20s - loss: 3.0623 - acc: 0.1661 - val_loss: 3.0078 - val_acc: 0.1750
Epoch 8/500
29190/29190 [=======================

In [31]:
### save model in between
model.fit(X, y, epochs=1, batch_size=512, callbacks = callbacks_list, verbose= True, validation_split=0.2)

Train on 29190 samples, validate on 7298 samples
Epoch 1/1
29190/29190 [==============================] - 14s - loss: 2.3087 - acc: 0.3286 - val_loss: 2.3075 - val_acc: 0.3352


In [32]:
model.fit(X, y, epochs=300, batch_size=512, verbose= True, validation_split=0.2)

Train on 29190 samples, validate on 7298 samples
Epoch 1/300
29190/29190 [==============================] - 14s - loss: 2.3102 - acc: 0.3266 - val_loss: 2.3032 - val_acc: 0.3298
Epoch 2/300
29190/29190 [==============================] - 14s - loss: 2.3125 - acc: 0.3252 - val_loss: 2.3035 - val_acc: 0.3328
Epoch 3/300
29190/29190 [==============================] - 13s - loss: 2.3142 - acc: 0.3244 - val_loss: 2.3104 - val_acc: 0.3320
Epoch 4/300
29190/29190 [==============================] - 13s - loss: 2.3090 - acc: 0.3269 - val_loss: 2.3018 - val_acc: 0.3356
Epoch 5/300
29190/29190 [==============================] - 14s - loss: 2.3131 - acc: 0.3251 - val_loss: 2.3045 - val_acc: 0.3334
Epoch 6/300
29190/29190 [==============================] - 13s - loss: 2.3041 - acc: 0.3237 - val_loss: 2.3028 - val_acc: 0.3383
Epoch 7/300
29190/29190 [==============================] - 14s - loss: 2.3012 - acc: 0.3281 - val_loss: 2.3064 - val_acc: 0.3320
Epoch 8/300
29190/29190 [=======================

In [33]:
### save model in between
model.fit(X, y, epochs=1, batch_size=512, callbacks = callbacks_list, verbose= True, validation_split=0.2)

Train on 29190 samples, validate on 7298 samples
Epoch 1/1
29190/29190 [==============================] - 13s - loss: 2.2387 - acc: 0.3440 - val_loss: 2.2718 - val_acc: 0.3489


In [34]:
model.fit(X, y, epochs=500, batch_size=512, verbose= True, validation_split=0.2)

Train on 29190 samples, validate on 7298 samples
Epoch 1/500
29190/29190 [==============================] - 13s - loss: 2.2445 - acc: 0.3439 - val_loss: 2.2695 - val_acc: 0.3478
Epoch 2/500
29190/29190 [==============================] - 13s - loss: 2.2424 - acc: 0.3452 - val_loss: 2.2667 - val_acc: 0.3437
Epoch 3/500
29190/29190 [==============================] - 13s - loss: 2.2406 - acc: 0.3459 - val_loss: 2.2654 - val_acc: 0.3442
Epoch 4/500
29190/29190 [==============================] - 13s - loss: 2.2327 - acc: 0.3441 - val_loss: 2.2758 - val_acc: 0.3430
Epoch 5/500
29190/29190 [==============================] - 13s - loss: 2.2413 - acc: 0.3428 - val_loss: 2.2702 - val_acc: 0.3461
Epoch 6/500
29190/29190 [==============================] - 13s - loss: 2.2381 - acc: 0.3435 - val_loss: 2.2644 - val_acc: 0.3467
Epoch 7/500
29190/29190 [==============================] - 13s - loss: 2.2380 - acc: 0.3427 - val_loss: 2.2632 - val_acc: 0.3460
Epoch 8/500
29190/29190 [=======================

In [35]:
model.fit(X, y, epochs=1, batch_size=512, callbacks = callbacks_list, verbose= True, validation_split=0.2)

Train on 29190 samples, validate on 7298 samples
Epoch 1/1
29190/29190 [==============================] - 13s - loss: 2.1840 - acc: 0.3575 - val_loss: 2.2378 - val_acc: 0.3574


In [36]:
model.fit(X, y, epochs=300, batch_size=512, verbose= True, validation_split=0.2)

Train on 29190 samples, validate on 7298 samples
Epoch 1/300
29190/29190 [==============================] - 13s - loss: 2.1780 - acc: 0.3628 - val_loss: 2.2343 - val_acc: 0.3557
Epoch 2/300
29190/29190 [==============================] - 13s - loss: 2.1803 - acc: 0.3575 - val_loss: 2.2335 - val_acc: 0.3553
Epoch 3/300
29190/29190 [==============================] - 13s - loss: 2.1899 - acc: 0.3595 - val_loss: 2.2405 - val_acc: 0.3502
Epoch 4/300
29190/29190 [==============================] - 14s - loss: 2.1851 - acc: 0.3604 - val_loss: 2.2252 - val_acc: 0.3583
Epoch 5/300
29190/29190 [==============================] - 13s - loss: 2.1865 - acc: 0.3590 - val_loss: 2.2382 - val_acc: 0.3548
Epoch 6/300
29190/29190 [==============================] - 13s - loss: 2.1800 - acc: 0.3596 - val_loss: 2.2380 - val_acc: 0.3520
Epoch 7/300
29190/29190 [==============================] - 13s - loss: 2.1849 - acc: 0.3600 - val_loss: 2.2398 - val_acc: 0.3542
Epoch 8/300
29190/29190 [=======================

In [37]:
model.fit(X, y, epochs=1, batch_size=512, callbacks = callbacks_list, verbose= True, validation_split=0.2)

Train on 29190 samples, validate on 7298 samples
Epoch 1/1
29190/29190 [==============================] - 14s - loss: 2.1760 - acc: 0.3626 - val_loss: 2.2294 - val_acc: 0.3594


In [38]:
model.fit(X, y, epochs=300, batch_size=512, verbose= True, validation_split=0.2)

Train on 29190 samples, validate on 7298 samples
Epoch 1/300
29190/29190 [==============================] - 14s - loss: 2.1686 - acc: 0.3665 - val_loss: 2.2300 - val_acc: 0.3574
Epoch 2/300
29190/29190 [==============================] - 14s - loss: 2.1646 - acc: 0.3658 - val_loss: 2.2291 - val_acc: 0.3608
Epoch 3/300
29190/29190 [==============================] - 13s - loss: 2.1606 - acc: 0.3672 - val_loss: 2.2253 - val_acc: 0.3586
Epoch 4/300
29190/29190 [==============================] - 14s - loss: 2.1617 - acc: 0.3675 - val_loss: 2.2242 - val_acc: 0.3593
Epoch 5/300
29190/29190 [==============================] - 13s - loss: 2.1812 - acc: 0.3580 - val_loss: 2.2379 - val_acc: 0.3541
Epoch 6/300
29190/29190 [==============================] - 13s - loss: 2.1650 - acc: 0.3654 - val_loss: 2.2303 - val_acc: 0.3587
Epoch 7/300
29190/29190 [==============================] - 14s - loss: 2.1522 - acc: 0.3686 - val_loss: 2.2248 - val_acc: 0.3561
Epoch 8/300
29190/29190 [=======================

In [39]:
model.fit(X, y, epochs=1, batch_size=512, callbacks = callbacks_list, verbose= True, validation_split=0.2)

Train on 29190 samples, validate on 7298 samples
Epoch 1/1
29190/29190 [==============================] - 13s - loss: 2.1391 - acc: 0.3730 - val_loss: 2.2169 - val_acc: 0.3643


In [40]:
predict_next_n(100)

Seed:  me...""
but i called my girl 
Generated: tee the tay i wanta be with you what i wanta be wiet you the way i like it 
averybody wou teet me th


In [41]:
model.fit(X, y, epochs=300, batch_size=512, verbose= True, validation_split=0.2)

Train on 29190 samples, validate on 7298 samples
Epoch 1/300
29190/29190 [==============================] - 13s - loss: 2.1459 - acc: 0.3689 - val_loss: 2.2192 - val_acc: 0.3646
Epoch 2/300
29190/29190 [==============================] - 14s - loss: 2.1287 - acc: 0.3748 - val_loss: 2.2092 - val_acc: 0.3649
Epoch 3/300
29190/29190 [==============================] - 13s - loss: 2.1385 - acc: 0.3714 - val_loss: 2.2170 - val_acc: 0.3601
Epoch 4/300
29190/29190 [==============================] - 13s - loss: 2.1467 - acc: 0.3670 - val_loss: 2.2193 - val_acc: 0.3620
Epoch 5/300
29190/29190 [==============================] - 14s - loss: 2.1485 - acc: 0.3677 - val_loss: 2.2102 - val_acc: 0.3653
Epoch 6/300
29190/29190 [==============================] - 14s - loss: 2.1585 - acc: 0.3676 - val_loss: 2.2188 - val_acc: 0.3624
Epoch 7/300
29190/29190 [==============================] - 14s - loss: 2.1406 - acc: 0.3704 - val_loss: 2.2169 - val_acc: 0.3685
Epoch 8/300
29190/29190 [=======================

In [42]:
model.fit(X, y, epochs=1, batch_size=512, callbacks = callbacks_list, verbose= True, validation_split=0.2)

Train on 29190 samples, validate on 7298 samples
Epoch 1/1
29190/29190 [==============================] - 14s - loss: 2.1220 - acc: 0.3736 - val_loss: 2.2066 - val_acc: 0.3731


In [43]:
predict_next_n(100)

Seed: can run to make it alright
i'l
Generated:  sou you aan toe that i wanta be with you what i wanta be with you what i wanta be with you what i w


In [ ]:
model.fit(X, y, epochs=300, batch_size=512, verbose= True, validation_split=0.2)

Train on 29190 samples, validate on 7298 samples
Epoch 1/300
29190/29190 [==============================] - 14s - loss: 2.1472 - acc: 0.3692 - val_loss: 2.2072 - val_acc: 0.3759
Epoch 2/300
29190/29190 [==============================] - 13s - loss: 2.1257 - acc: 0.3770 - val_loss: 2.2118 - val_acc: 0.3717
Epoch 3/300
29190/29190 [==============================] - 13s - loss: 2.1344 - acc: 0.3766 - val_loss: 2.2114 - val_acc: 0.3702
Epoch 4/300
29190/29190 [==============================] - 13s - loss: 2.1483 - acc: 0.3690 - val_loss: 2.1998 - val_acc: 0.3746
Epoch 5/300
29190/29190 [==============================] - 13s - loss: 2.1298 - acc: 0.3723 - val_loss: 2.2062 - val_acc: 0.3667
Epoch 6/300
29190/29190 [==============================] - 13s - loss: 2.1330 - acc: 0.3769 - val_loss: 2.1946 - val_acc: 0.3765
Epoch 7/300
29190/29190 [==============================] - 13s - loss: 2.1315 - acc: 0.3709 - val_loss: 2.2052 - val_acc: 0.3752
Epoch 8/300
29190/29190 [=======================

In [ ]:
predict_next_n(100)

In [ ]:
model.fit(X, y, epochs=300, batch_size=512, verbose= True, validation_split=0.2)

In [ ]:
model.fit(X, y, epochs=1, batch_size=512, callbacks = callbacks_list, verbose= True, validation_split=0.2)

In [ ]:
predict_next_n(100)

In [ ]:
model.evaluate(X,y)

172192/681794 [======>.......................] - ETA: 1369s

In [ ]:
print_time(start_time_0)